In [2]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import glob
import os
import scipy
from astropy.io import fits
import importlib.util
import matplotlib.pyplot as plt
from sklearn import metrics
import matplotlib.backends.backend_pdf
import re
import copy
from astropy.visualization import make_lupton_rgb
import matplotlib.image as mpimg
import shutil
import wget
import urllib.request

In [3]:
def fluxToMag(f):
    return (-2.5 * (np.log(f)/np.log(10.) - 9))

In [10]:
print(fluxToMag(7.807234))
print('min flux -> max mag',fluxToMag(0.000167))
print('max flux -> min mag',fluxToMag(1.380388))

20.2687570092
min flux -> max mag 31.9432088221
max flux -> min mag 22.1499970619


In [2]:
DR = 7
t_folder = '001'
t_url = 'http://portal.nersc.gov/project/cosmo/data/legacysurvey/dr{}/tractor/{}/'.format(DR, t_folder) # DR7
# 'http://portal.nersc.gov/project/cosmo/data/legacysurvey/dr7/tractor/001/'
foldername = '/Users/mac/Desktop/LBNL/DR{}/tractor_folders/tractor_{}'.format(DR, t_folder) # DR7
print('t_folder:', t_folder, '  type=', type(t_folder))
print('t_url = ', t_url)
print('foldername = ', foldername)
t_folder_file = wget.download(t_url, foldername) #, foldername
with open(t_folder_file) as wgetfile:
    wgetdata = wgetfile.read()
os.remove(foldername)

t_folder: 001   type= <class 'str'>
t_url =  http://portal.nersc.gov/project/cosmo/data/legacysurvey/dr7/tractor/001/
foldername =  /Users/mac/Desktop/LBNL/DR7/tractor_folders/tractor_001


In [3]:
wgetdata

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<html>\n <head>\n  <title>Index of /project/cosmo/data/legacysurvey/dr7/tractor/001</title>\n </head>\n <body>\n<h1>Index of /project/cosmo/data/legacysurvey/dr7/tractor/001</h1>\n  <table>\n   <tr><th valign="top"><img src="/icons/blank.gif" alt="[ICO]"></th><th><a href="?C=N;O=D">Name</a></th><th><a href="?C=M;O=A">Last modified</a></th><th><a href="?C=S;O=A">Size</a></th><th><a href="?C=D;O=A">Description</a></th></tr>\n   <tr><th colspan="5"><hr></th></tr>\n<tr><td valign="top"><img src="/icons/back.gif" alt="[PARENTDIR]"></td><td><a href="/project/cosmo/data/legacysurvey/dr7/tractor/">Parent Directory</a></td><td>&nbsp;</td><td align="right">  - </td><td>&nbsp;</td></tr>\n<tr><td valign="top"><img src="/icons/unknown.gif" alt="[   ]"></td><td><a href="legacysurvey_dr7_tractor_001.sha256sum">legacysurvey_dr7_tractor_001.sha256sum</a></td><td align="right">2018-08-02 10:27  </td><td align="right"> 66K</td><td>&nbsp;</td></tr>

In [4]:
prog = re.compile('(?<=\.fits">).{21}')
result = prog.findall(wgetdata)
print(type(result), len(result))

<class 'list'> 768


In [5]:
result[0]

'tractor-0010p292.fits'

In [18]:
filelink = t_url + result[100]
t_file = wget.download(filelink, '/Users/mac/Desktop/LBNL/DR{}/tractor_fits/'.format(DR))
with fits.open(t_file) as fit:
    filedata = fit[1].data
os.remove(t_file)

In [19]:
filedata.shape[0]

3691

In [30]:
for i in range(len(filedata[1000])):
    print(i, ':   ', filedata[1000][i])

0 :    7000
1 :    370624
2 :    0011p070
3 :    1000
4 :    True
5 :    False
6 :    REX
7 :    1.01328571176
8 :    6.93761025116
9 :    1.21723e+09
10 :    1.22391e+09
11 :    3430.58
12 :    942.442
13 :    [ 102.31381226  116.11844635  116.54226685  118.30312347  109.39228058]
14 :    0.0439843
15 :    56884.3977593
16 :    58018.2268845
17 :    
18 :    0
19 :    0.0
20 :    0.0
21 :    0.0
22 :    0.0
23 :    0.0
24 :    0.0
25 :    0.0
26 :    False
27 :    0.0
28 :    0.0
29 :    0
30 :    0.0
31 :    0.0
32 :    0
33 :    0.0
34 :    0.0
35 :    0
36 :    False
37 :    0.0
38 :    0.0
39 :    0
40 :    0
41 :    0.0
42 :    False
43 :    0.0
44 :    0.281477
45 :    0.397728
46 :    0.0
47 :    0.852714
48 :    0.0
49 :    -0.462128
50 :    -0.638033
51 :    43.2906
52 :    -6.74302
53 :    0.0
54 :    731.085
55 :    251.356
56 :    0.0
57 :    38.9229
58 :    0.0
59 :    2.85036
60 :    0.546787
61 :    0.000800532
62 :    1.77671e-05
63 :    0.0
64 :    0.123338
65 :    0.

In [44]:
i = 20
nobs_g = filedata[i][104] # DR7
nobs_r = filedata[i][105]
nobs_z = filedata[i][107]
mtype = filedata[i][6]
print(nobs_g, nobs_r, nobs_z, mtype)

2 2 2 REX


In [51]:
layer = 'decals-dr7'
ra, dec = (1.00810635927,-0.377373797325)
jpeg_folder = '/Users/mac/Desktop/LBNL/DR7/jpegs_dr7_NONLENS_CANDIDATES/'
jpeg = urllib.request.urlretrieve('http://legacysurvey.org/viewer/jpeg-cutout?ra={}&dec={}&layer={}&pixscale=0.262&bands=grz&size=101'.format(ra, dec, layer), jpeg_folder+'TEST.jpeg')                              
                                  